### Intertidal Analysis

In [ ]:
# import libraries

# basic df
import pandas as pd
import numpy as np

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# encoding
from sklearn.preprocessing import LabelEncoder

# modelling
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn import tree
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

# map
import folium
from folium.plugins import HeatMap

#### Importing Data Sources

In [ ]:
# intertidal coordinates
intertidal_coords = pd.read_excel('C:\\Users\\izzyc\\Downloads\\IntertidalCoordinates (1).xlsx', sheet_name='Sheet1')
# condition
intertidal_condition = pd.read_excel("C:\\Users\\izzyc\\Downloads\\Intertidal_Oyster_Condition_Dec_2019-July_2023.xlsx", sheet_name='Intertidal_Condition')
# water quality
intertidal_wq = pd.read_excel("C:\\Users\\izzyc\\Downloads\\Intertidal_Oyster_Condition_WQ_Dec_2019-July_2023.xlsx", sheet_name='Water_Quality')
# disease
intertidal_disease = pd.read_excel("C:\\Users\\izzyc\\Downloads\\Interdital_Oyster_Disease_July_2021-February_2023.xlsx", sheet_name = 'Disease_Prevelance')


#### Preprocessing Data Sources

In [ ]:
# coordinates
coords = intertidal_coords


# condition
intertidal_condition['SiteID'] = intertidal_condition['Site'].astype(str) + '-' + intertidal_condition['Reef'].astype(str)
condition_voi = ['SiteID','Reef','Date_Sample', 'Oyster_Number', 'Height_mm', 'Length_mm', 'Width_mm',
                 'Total_Weight_g', 'Soft_Tiss_Wet', 'Shell_Wet', 'Soft_Tiss_Dry', 'Shell_Dry', 'Sex']
condition = intertidal_condition[condition_voi]


# water quality
intertidal_wq['SiteID'] = intertidal_wq['Site'].astype(str) + '-' + intertidal_wq['Reef'].astype(str)
wq_voi = ['SiteID', 'Date_Collected', 'Reef', 'Time', 'Ambient_Temp_°C', 'Water_Temp_°C'
          ,'Salinity_PPT' ]
wq = intertidal_wq[wq_voi]

# disease
intertidal_disease['SiteID'] = intertidal_disease['Site'].astype(str) + '-' + intertidal_disease['Reef'].astype(str)
disease_voi = ['SiteID', 'Reef','Date_Collected', 'Oyster_Number', 'Disease_Scale#']
disease = intertidal_disease[disease_voi]

#### Building Dataset with Variables of Interest

In [ ]:

# Step one. Merge coord and condition on SiteID
merged_df = pd.merge(condition, coords, on='SiteID')

# Step two. Merge with water quality on siteID, date collected
merged_df = pd.merge(merged_df, wq, left_on=['SiteID', 'Date_Sample'], right_on=['SiteID', 'Date_Collected'])

# Step three. Merge with disease on SiteID, Date Collected, Oyster Number
merged_df = pd.merge(merged_df, disease, on=['SiteID', 'Oyster_Number'])

# Drop duplicate columns (if any)
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]

# Display the resulting dataframe
merged_df.info()




In [ ]:
merged_df = merged_df.drop(columns=[ 'Reef_y', 'Site'])
merged_df['Date_Collected'] = merged_df['Date_Collected_y']
merged_df['Disease_Scale'] = merged_df['Disease_Scale#']


##### Cleaning: Converting Objects to Float and Datetime

In [ ]:
# convert necessary attributes to float
merged_df['Height_mm'] = pd.to_numeric(merged_df['Height_mm'], errors='coerce')
merged_df['Length_mm'] = pd.to_numeric(merged_df['Length_mm'], errors='coerce')
merged_df['Width_mm'] = pd.to_numeric(merged_df['Width_mm'], errors='coerce')
merged_df['Total_Weight_g'] = pd.to_numeric(merged_df['Total_Weight_g'], errors='coerce')
merged_df['Soft_Tiss_Wet'] = pd.to_numeric(merged_df['Soft_Tiss_Wet'], errors='coerce')
merged_df['Shell_Wet'] = pd.to_numeric(merged_df['Shell_Wet'], errors='coerce')
merged_df['Soft_Tiss_Dry'] = pd.to_numeric(merged_df['Soft_Tiss_Dry'], errors='coerce')
merged_df['Shell_Dry'] = pd.to_numeric(merged_df['Shell_Dry'], errors='coerce')
merged_df['Ambient_Temp_°C'] = pd.to_numeric(merged_df['Ambient_Temp_°C'], errors='coerce')
merged_df['Water_Temp_°C'] = pd.to_numeric(merged_df['Water_Temp_°C'], errors='coerce')
merged_df['Salinity_PPT'] = pd.to_numeric(merged_df['Salinity_PPT'], errors='coerce')
merged_df['Disease_Scale'] = pd.to_numeric(merged_df['Disease_Scale'], errors='coerce')

# now convert time and date_collected to datetime
# first replace ND with naT (not a time)
merged_df['Date_Collected'] = merged_df['Date_Collected'].replace('ND', pd.NaT)
merged_df['Date_Sample'] = merged_df['Date_Sample'].replace('ND', pd.NaT)
merged_df['Time'] = merged_df['Time'].replace('ND', pd.NaT)
merged_df['Date_Collected'] = pd.to_datetime(merged_df['Date_Collected'], errors='coerce')
merged_df['Time'] = pd.to_datetime(merged_df['Time'], errors='coerce')
merged_df['Date_Sample'] = pd.to_datetime(merged_df['Date_Sample'], errors='coerce')

##### Encoding Categorical to Levels

In [ ]:
# finally, encode SiteID and sex to levels

label_encoder = LabelEncoder()

# transform site and sex
merged_df['SiteID_encoded'] = label_encoder.fit_transform(merged_df['SiteID'])
merged_df['Sex_encoded'] = label_encoder.fit_transform(merged_df['Sex'])


In [ ]:
merged_df.drop(columns=['Disease_Scale#', 'Reef'])
merged_df.info()

In [ ]:
merged_df['Disease_Scale'].describe()

#### Identifying Missing Data

In [ ]:
# Create a heatmap of missing values
plt.figure(figsize=(10, 6))
sns.heatmap(merged_df.isnull(), cmap='viridis', cbar=False, yticklabels=False)

# Set plot title
plt.title('Missing Values in the Dataset')

# Display the plot
plt.show()


#### Correlation Matrix

In [ ]:
# Correlation Matrix
correlation_matrix = merged_df.corr()

# Seaborn heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Matrix of Predictors')
plt.show()

#### Time vs. Size

In [ ]:
# visualize size over time per site
# visualize height over time
plt.figure(figsize=(10,6))
plt.scatter(merged_df['Date_Sample'], merged_df['Height_mm'], c = merged_df['SiteID_encoded'], cmap = 'viridis')
plt.colorbar(label='SiteID')
plt.xlabel('Date Sampled')
plt.ylabel('Height in mm')
plt.title('Oyster Height over time')
plt.grid(True)

plt.tight_layout()
plt.show()

#### Size By SiteID

In [ ]:
# distribution of size per site
plt.figure(figsize=(10, 6))
sns.violinplot(x='SiteID', y='Height_mm', data=merged_df, palette='Set2')
plt.xlabel('Site ID')
plt.ylabel('Height (mm)')
plt.title('Violin Plot of Height by Site')
plt.grid(True)

# Show the plot
plt.tight_layout()
plt.show()

#### Seasonal Salinity and Water Temp 

In [ ]:
plt.figure(figsize=(12,6))
plt.scatter(merged_df['Date_Sample'], merged_df['Salinity_PPT'], marker='o' )       # replace with water temp for alternate viz
plt.xlabel('Sample Date')
plt.ylabel('Salinity PPT')
plt.grid(True)


plt.tight_layout()
plt.show()

#### Creating Binary Indicator of Dermo for Prediction Modelling

In [ ]:
# Create a new variable 'has_dermo' based on the condition
merged_df['has_dermo'] = (merged_df['Disease_Scale'] != 0).astype(int)
# Count the number of observations with 'dermo'
dermo_count = merged_df['has_dermo'].sum()

print(dermo_count)




#### Decision Tree Classifier

In [ ]:
# define features and target
features = [ 'Oyster_Number', 'Height_mm', 'Length_mm', 'Width_mm', 'Total_Weight_g',
            'Soft_Tiss_Wet', 'Shell_Wet', 'Soft_Tiss_Dry', 'Shell_Dry', 'POINT_X', 'POINT_Y',
            'Ambient_Temp_°C', 'Water_Temp_°C', 'Salinity_PPT', 'SiteID_encoded', 'Sex_encoded']

target = 'has_dermo'

X = merged_df[features]
y = merged_df[target]

imputer = SimpleImputer(strategy='mean')  # replace nan with mean
X_imputed = imputer.fit_transform(X)

# train test split: use 80-20
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)


# make decision tree
max_depth_value = 5  # implementing max depth for second iteration
model = DecisionTreeClassifier(max_depth=max_depth_value, random_state=42)
model.fit(X_train, y_train)

# make predictions on test set
y_pred = model.predict(X_test)

# evalutation
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", classification_rep)

# visualize tree
plt.figure(figsize=(12, 8))
tree.plot_tree(model, feature_names=features, class_names=['Not Hit', 'Hit'], filled=True, rounded=True)
plt.show()

##### Feature Importance for Decision Tree

In [ ]:
# Get feature importances from the model
feature_importances = model.feature_importances_

# Create a dataframe with feature names and their importances
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

# Sort the dataframe by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Plot the feature importances
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df, palette='viridis')
plt.title('Feature Importances in Decision Tree Classifier')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

#### Neural Net Classifier

In [ ]:
# standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# create a neural network model
model = Sequential()
model.add(Dense(30, activation='relu', input_dim=X_train_scaled.shape[1]))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid')) 

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.1)

# evaluate the model on the test set
y_pred_prob = model.predict(X_test_scaled)
y_pred = (y_pred_prob > 0.5).astype(int)

# evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", classification_rep)

#### Optimizing Hyperparams

In [ ]:

def optimize_neural_network(X, y, param_grid, test_size=0.2, random_state=42):
    """
    Optimize parameters for a neural network classifier using grid search.

    Parameters:
    - X: Features
    - y: Target variable
    - param_grid: Dictionary of hyperparameters for grid search
    - test_size: Proportion of the dataset to include in the test split
    - random_state: Seed for random number generation

    Returns:
    - Best parameters found by grid search
    - Best model with the optimal parameters
    - Classification report on the test set using the best model
    """


    # create a neural network classifier
    neural_net = MLPClassifier()

    # create a GridSearchCV object
    grid_search = GridSearchCV(neural_net, param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)

    # fit the grid search to the data
    grid_search.fit(X_train, y_train)

    # get the best parameters and the corresponding best model
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    # evaluate the best model on the test set
    y_pred = best_model.predict(X_test)
    report = classification_report(y_test, y_pred)

    return best_params, best_model, report

# usage:
param_grid = {'hidden_layer_sizes': [(10,), (50,), (100,)], 'alpha': [0.0001, 0.001, 0.01]}
best_params, best_model, classification_report = optimize_neural_network(X, y, param_grid)
print("Best Parameters:", best_params)
print("Classification Report:\n", classification_report)


#### Naive Bayes Classifier

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

# Gaussian Naive Bayes classifier
naive_bayes = GaussianNB()

# train
naive_bayes.fit(X_train, y_train)

# predict
y_pred = naive_bayes.predict(X_test)

# evaluate
report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)


#### Map

In [ ]:
# check for NaN values in 'disease_scale' and drop those rows
merged_df = merged_df.dropna(subset=['Disease_Scale'])

# convert 'disease_scale' to numeric 
merged_df['Disease_Scale'] = pd.to_numeric(merged_df['Disease_Scale'], errors='coerce')

# create a base map centered around the average latitude and longitude
average_lat = merged_df['POINT_Y'].mean()
average_lon = merged_df['POINT_X'].mean()
mymap = folium.Map(location=[average_lat, average_lon], zoom_start=10)

# create a HeatMap layer
heat_data = [[row['POINT_Y'], row['POINT_X'], row['Disease_Scale']] for index, row in merged_df.iterrows()]
HeatMap(heat_data, name="Disease Scale").add_to(mymap)

# display the map
mymap.save("disease_map.html")
